In [43]:
from ultralytics import YOLO, solutions
import cv2
import os
import numpy as np

In [5]:
model = YOLO('yolo11n.pt')

In [8]:
def display_objects(video_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            label = model.names[cls]
            conf = box.conf[0]

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [53]:
def counting_objects_in_regions(video_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    regions = [(500, 300, 900, 900),(1200, 300, 1600, 900)]
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]
        map_count_in_region = {}
        for index, region in enumerate(regions):
            map_count_in_region[index] = 0
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            label = model.names[cls]
            conf = box.conf[0]

            # Tính tâm của bounding box
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            color = (0, 0, 255)
            # Kiểm tra tâm nằm trong vùng định sẵn
            for index, region in enumerate(regions):
                rx1, ry1, rx2, ry2 = region
                if rx1 <= cx <= rx2 and ry1 <= cy <= ry2:
                    map_count_in_region[index] = map_count_in_region[index] +1
                    color = (0, 255, 0)  # xanh lá nếu nằm trong vùng


            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

         # Vẽ vùng region
        for index, region in enumerate(regions):
            cv2.rectangle(frame, (region[0], region[1]), (region[2], region[3]), (255, 255, 0), 2)
            cv2.putText(frame, f'Count in region: {map_count_in_region[index]}', (region[0], region[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv2.putText(frame, 'prompting: Please detect object in my regions', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [54]:
counting_objects_in_regions("../resources/video.mp4", "../resources/counting_object_regions.mp4")


0: 384x640 1 car, 55.0ms
Speed: 2.8ms preprocess, 55.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 37.4ms
Speed: 1.4ms preprocess, 37.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 34.8ms
Speed: 1.4ms preprocess, 34.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 34.0ms
Speed: 1.6ms preprocess, 34.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 38.9ms
Speed: 2.0ms preprocess, 38.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 31.2ms
Speed: 1.6ms preprocess, 31.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 trucks, 31.6ms
Speed: 1.5ms preprocess, 31.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 32.8ms
Speed: 1.4ms preprocess, 32.8ms inference

In [29]:
def counting_objects_throw_line(video_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            label = model.names[cls]
            conf = box.conf[0]

            cy = (y1 + y2) // 2

            color = (0, 0, 255)
            if cy < 300:
                color = (0, 255, 0)


            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        cv2.line(frame, (10, 300), (1800, 300), (0, 255, 0), 2)
         # Vẽ vùng region
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [59]:
counting_objects_throw_line("../resources/video.mp4", "../resources/counting_object_line.mp4")


0: 384x640 1 car, 48.3ms
Speed: 2.3ms preprocess, 48.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 41.6ms
Speed: 1.7ms preprocess, 41.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 48.2ms
Speed: 1.5ms preprocess, 48.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 36.7ms
Speed: 1.6ms preprocess, 36.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 39.2ms
Speed: 2.0ms preprocess, 39.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 34.1ms
Speed: 1.6ms preprocess, 34.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 trucks, 34.3ms
Speed: 1.3ms preprocess, 34.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 34.1ms
Speed: 1.3ms preprocess, 34.1ms inference

In [40]:
def counting_objects_in_regions_and_crop(video_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    regions = [(500, 300, 900, 900),(1200, 300, 1600, 900)]
    crop_id = 0
    cropped_ids = set()
    crop_dir = "crops"
    os.makedirs(crop_dir, exist_ok=True)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model.track(frame, persist=True, tracker="bytetrack.yaml")[0]
        # results = model(frame)[0]
        map_count_in_region = {}
        for index, region in enumerate(regions):
            map_count_in_region[index] = 0
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            label = model.names[cls]
            conf = box.conf[0]

            # Tracking ID
            if hasattr(box, 'id') and box.id is not None:
                track_id = int(box.id[0])
            else:
                continue  # Skip nếu không có ID

            # Tính tâm của bounding box
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            color = (0, 0, 255)
            # Kiểm tra tâm nằm trong vùng định sẵn
            for index, region in enumerate(regions):
                rx1, ry1, rx2, ry2 = region
                if rx1 <= cx <= rx2 and ry1 <= cy <= ry2:
                    map_count_in_region[index] = map_count_in_region[index] +1
                    color = (0, 255, 0)  # xanh lá nếu nằm trong vùng
                          # 👉 CROP và LƯU object
                    if track_id not in cropped_ids:
                        obj_crop = frame[y1:y2, x1:x2]
                        crop_path = os.path.join(crop_dir, f"object_{crop_id:04d}_{label}.jpg")
                        cv2.imwrite(crop_path, obj_crop)
                        cropped_ids.add(track_id)
                        crop_id += 1


            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

         # Vẽ vùng region
        for index, region in enumerate(regions):
            cv2.rectangle(frame, (region[0], region[1]), (region[2], region[3]), (255, 255, 0), 2)
            cv2.putText(frame, f'Count in region: {map_count_in_region[index]}', (region[0], region[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [56]:
def counting_objects_in_regions_and_crop_and_blur(video_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    regions = [(500, 300, 900, 900),(1200, 300, 1600, 900)]
    crop_id = 0
    cropped_ids = set()
    crop_dir = "crops"
    os.makedirs(crop_dir, exist_ok=True)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model.track(frame, persist=True, tracker="bytetrack.yaml")[0]
        # results = model(frame)[0]
        map_count_in_region = {}
        for index, region in enumerate(regions):
            map_count_in_region[index] = 0
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            label = model.names[cls]
            conf = box.conf[0]

            # Tracking ID
            if hasattr(box, 'id') and box.id is not None:
                track_id = int(box.id[0])
            else:
                continue  # Skip nếu không có ID

            # Tính tâm của bounding box
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            color = (0, 0, 255)
            # Kiểm tra tâm nằm trong vùng định sẵn
            for index, region in enumerate(regions):
                rx1, ry1, rx2, ry2 = region
                if rx1 <= cx <= rx2 and ry1 <= cy <= ry2:
                    map_count_in_region[index] = map_count_in_region[index] +1
                    color = (0, 255, 0)  # xanh lá nếu nằm trong vùng
                          # 👉 CROP và LƯU object

                     # ✅ Blur object trong frame
                    object_area = frame[y1:y2, x1:x2]
                    blurred = cv2.GaussianBlur(object_area, (35, 35), 30)
                    frame[y1:y2, x1:x2] = blurred

                    # if track_id not in cropped_ids:
                    #     obj_crop = frame[y1:y2, x1:x2]
                    #     crop_path = os.path.join(crop_dir, f"object_{crop_id:04d}_{label}.jpg")
                    #     cv2.imwrite(crop_path, obj_crop)
                    #     cropped_ids.add(track_id)
                    #     crop_id += 1


            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

         # Vẽ vùng region
        for index, region in enumerate(regions):
            cv2.rectangle(frame, (region[0], region[1]), (region[2], region[3]), (255, 255, 0), 2)
            cv2.putText(frame, f'Count in region: {map_count_in_region[index]}', (region[0], region[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv2.putText(frame, 'prompting: Please detect object in my regions and blur it', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [57]:
counting_objects_in_regions_and_crop_and_blur("../resources/video.mp4", "../resources/counting_object_regions_blur.mp4")


0: 384x640 1 car, 43.3ms
Speed: 3.0ms preprocess, 43.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 52.0ms
Speed: 2.3ms preprocess, 52.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 39.5ms
Speed: 1.6ms preprocess, 39.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 35.1ms
Speed: 1.5ms preprocess, 35.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 35.3ms
Speed: 1.9ms preprocess, 35.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 35.3ms
Speed: 1.2ms preprocess, 35.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 32.5ms
Speed: 1.3ms preprocess, 32.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 33.5ms
Speed: 1.3ms preprocess, 33.5ms inference

In [46]:
def show_objects_heatmap(video_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # Bản đồ tích lũy điểm nóng
    heatmap_accumulator = np.zeros((h, w), dtype=np.float32)

    # Kernel để tạo hiệu ứng lan tỏa quanh centroid
    kernel = np.zeros((50, 50), np.uint8)
    cv2.circle(kernel, (25, 25), 25, 1, -1)
    ALPHA = 0.7  # độ trong suốt của heatmap overlay
    DECAY = 0.98  # làm mờ dần heatmap theo thời gian
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

             # Chồng kernel vào heatmap tại vị trí centroid
            top = max(cy - 25, 0)
            left = max(cx - 25, 0)
            bottom = min(cy + 25, h)
            right = min(cx + 25, w)

            h_k = bottom - top
            w_k = right - left

            heatmap_accumulator[top:bottom, left:right] += kernel[:h_k, :w_k]

        # Làm mờ heatmap qua thời gian
        heatmap_accumulator *= DECAY

        # Normalize và chuyển sang màu
        norm_heatmap = cv2.normalize(heatmap_accumulator, None, 0, 255, cv2.NORM_MINMAX)
        heatmap_color = cv2.applyColorMap(norm_heatmap.astype(np.uint8), cv2.COLORMAP_JET)

         # Overlay heatmap lên frame gốc
        blended = cv2.addWeighted(frame, 1 - ALPHA, heatmap_color, ALPHA, 0)
        out.write(blended)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [50]:
# display_objects("../resources/video.mp4", "output_video.mp4")
counting_objects_in_regions("../resources/video.mp4", "../resources/counting_object_regions.mp4")


0: 384x640 1 car, 45.8ms
Speed: 9.7ms preprocess, 45.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 34.8ms
Speed: 1.4ms preprocess, 34.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 33.7ms
Speed: 1.6ms preprocess, 33.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 34.5ms
Speed: 1.3ms preprocess, 34.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 37.9ms
Speed: 1.5ms preprocess, 37.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 32.2ms
Speed: 1.3ms preprocess, 32.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 trucks, 33.2ms
Speed: 1.5ms preprocess, 33.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 32.7ms
Speed: 1.5ms preprocess, 32.7ms inference